In [2]:
# Loading the data set
from datasets import load_dataset
ds = load_dataset("imagefolder",data_dir='training_set')

Resolving data files:   0%|          | 0/22620 [00:00<?, ?it/s]

In [3]:
data = ds["train"].train_test_split(test_size=0.15)
data["train"][400]["label"]


12

In [4]:
labels = data["train"].features['label']
labels

ClassLabel(names=['Asian Green Bee-Eater', 'Brown-Headed Barbet', 'Cattle Egret', 'Common Kingfisher', 'Common Myna', 'Common Rosefinch', 'Common Tailorbird', 'Coppersmith Barbet', 'Forest Wagtail', 'Gray Wagtail', 'Hoopoe', 'House Crow', 'Indian Grey Hornbill', 'Indian Peacock', 'Indian Pitta', 'Indian Roller', 'Jungle Babbler', 'Northern Lapwing', 'Red-Wattled Lapwing', 'Ruddy Shelduck', 'Rufous Treepie', 'Sarus Crane', 'White Wagtail', 'White-Breasted Kingfisher', 'White-Breasted Waterhen'], id=None)

In [5]:
labels.int2str(data["train"][400]["label"])

'Indian Grey Hornbill'

In [6]:
from datasets import load_metric
metric = load_metric("accuracy")

C:\Users\harsh\AppData\Local\Temp\ipykernel_6452\3366162530.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
c:\PF\Projects\env\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [41]:
labels = data["train"].features["label"].names
label2id,id2label =dict(),dict()
for i,label in enumerate(labels):
    label2id[label] = i
    id2label[i]=label

In [8]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
from transformers import AutoImageProcessor

model_name_or_path ="dennisjooo/Birds-Classifier-EfficientNetB2"
feature_extractor = AutoImageProcessor.from_pretrained(model_name_or_path)

In [10]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)
from transformers import AutoImageProcessor

model_name_or_path = "dennisjooo/Birds-Classifier-EfficientNetB2"
feature_extractor = AutoImageProcessor.from_pretrained(model_name_or_path)

# Use the size specified in the model name (224)
image_size = 224

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)

train_transforms = Compose(
    [
        RandomResizedCrop(image_size),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize,
    ]
)

val_transforms = Compose(
    [
        Resize(image_size),
        CenterCrop(image_size),
        ToTensor(),
        normalize,
    ]
)

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [
        val_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

In [11]:
train_ds = data["train"]
val_ds = data["test"]

In [12]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [14]:
from transformers import AutoModelForImageClassification
model_name_or_path = 'dennisjooo/Birds-Classifier-EfficientNetB2'
model = AutoModelForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): labels[i] for i in range(len(labels))},
    label2id={labels[i]: str(i) for i in range(len(labels))},
    ignore_mismatched_sizes=True
)

Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at dennisjooo/Birds-Classifier-EfficientNetB2 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([525]) in the checkpoint and torch.Size([25]) in the model instantiated
- classifier.weight: found shape torch.Size([525, 1408]) in the checkpoint and torch.Size([25, 1408]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    'finetuned-birds-classifier',  # Adjust the name to reflect your project
    per_device_train_batch_size=16,  # Adjust if needed based on GPU memory
    evaluation_strategy="steps",  # Or "epoch" if you'd prefer end-of-epoch evaluation
    num_train_epochs=4,
    fp16=True,
    save_steps=500,  # Adjust based on your dataset size
    eval_steps=500,  # Adjust based on your dataset size
    logging_steps=50,  # Adjust based on how often you want logging
    learning_rate=2e-5,  # Lower the learning rate for fine-tuning
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,  # Set to True if you want to push to Hugging Face Hub
    report_to='tensorboard',  # Enable TensorBoard for logging
    load_best_model_at_end=True,
    hub_strategy="end"  # Use "every_save" if you want intermediate saves to Hub
)


c:\PF\Projects\env\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
import numpy as np

# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [18]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

  0%|          | 0/4808 [00:00<?, ?it/s]

{'loss': 3.2895, 'grad_norm': 5.715845108032227, 'learning_rate': 1.979201331114809e-05, 'epoch': 0.04}
{'loss': 3.2499, 'grad_norm': 5.686153411865234, 'learning_rate': 1.9584026622296175e-05, 'epoch': 0.08}
{'loss': 3.1352, 'grad_norm': 5.594604015350342, 'learning_rate': 1.9376039933444262e-05, 'epoch': 0.12}
{'loss': 3.0819, 'grad_norm': 5.311897277832031, 'learning_rate': 1.916805324459235e-05, 'epoch': 0.17}
{'loss': 3.0072, 'grad_norm': 5.5024333000183105, 'learning_rate': 1.8960066555740436e-05, 'epoch': 0.21}
{'loss': 2.9131, 'grad_norm': 5.361512660980225, 'learning_rate': 1.8752079866888523e-05, 'epoch': 0.25}
{'loss': 2.858, 'grad_norm': 5.253929615020752, 'learning_rate': 1.8544093178036607e-05, 'epoch': 0.29}
{'loss': 2.77, 'grad_norm': 5.410942077636719, 'learning_rate': 1.8336106489184694e-05, 'epoch': 0.33}
{'loss': 2.6997, 'grad_norm': 5.648326396942139, 'learning_rate': 1.812811980033278e-05, 'epoch': 0.37}
{'loss': 2.616, 'grad_norm': 5.3726935386657715, 'learning_r

  0%|          | 0/425 [00:00<?, ?it/s]

{'eval_loss': 2.340571403503418, 'eval_accuracy': 0.5655761862658414, 'eval_runtime': 117.3046, 'eval_samples_per_second': 28.925, 'eval_steps_per_second': 3.623, 'epoch': 0.42}
{'loss': 2.5459, 'grad_norm': 5.260122776031494, 'learning_rate': 1.7712146422628954e-05, 'epoch': 0.46}
{'loss': 2.4942, 'grad_norm': 5.387138843536377, 'learning_rate': 1.750415973377704e-05, 'epoch': 0.5}
{'loss': 2.4061, 'grad_norm': 5.292346477508545, 'learning_rate': 1.7296173044925125e-05, 'epoch': 0.54}
{'loss': 2.3804, 'grad_norm': 5.237049102783203, 'learning_rate': 1.7088186356073212e-05, 'epoch': 0.58}
{'loss': 2.2565, 'grad_norm': 5.329747676849365, 'learning_rate': 1.68801996672213e-05, 'epoch': 0.62}
{'loss': 2.2161, 'grad_norm': 5.015501499176025, 'learning_rate': 1.6672212978369386e-05, 'epoch': 0.67}
{'loss': 2.1249, 'grad_norm': 5.067910671234131, 'learning_rate': 1.6464226289517473e-05, 'epoch': 0.71}
{'loss': 2.0471, 'grad_norm': 4.882927417755127, 'learning_rate': 1.625623960066556e-05, 'e

  0%|          | 0/425 [00:00<?, ?it/s]

...

{'eval_loss': 0.4659765362739563, 'eval_accuracy': 0.8912466843501327, 'eval_runtime': 95.2831, 'eval_samples_per_second': 35.61, 'eval_steps_per_second': 4.46, 'epoch': 2.91}
{'loss': 0.9658, 'grad_norm': 3.209306240081787, 'learning_rate': 5.237104825291181e-06, 'epoch': 2.95}
{'loss': 0.9958, 'grad_norm': 2.9743523597717285, 'learning_rate': 5.029118136439268e-06, 'epoch': 3.0}
{'loss': 0.9595, 'grad_norm': 3.174795627593994, 'learning_rate': 4.821131447587355e-06, 'epoch': 3.04}
{'loss': 0.9679, 'grad_norm': 3.3084464073181152, 'learning_rate': 4.613144758735441e-06, 'epoch': 3.08}
{'loss': 0.9595, 'grad_norm': 3.7894246578216553, 'learning_rate': 4.405158069883527e-06, 'epoch': 3.12}
{'loss': 0.9417, 'grad_norm': 3.385998010635376, 'learning_rate': 4.201331114808653e-06, 'epoch': 3.16}
{'loss': 0.9332, 'grad_norm': 4.913998603820801, 'learning_rate': 3.993344425956739e-06, 'epoch': 3.2}
{'loss': 0.8914, 'grad_norm': 3.020223379135132, 'learning_rate': 3.7853577371048255e-06, 'epoc

  0%|          | 0/425 [00:00<?, ?it/s]

{'eval_loss': 0.3981543183326721, 'eval_accuracy': 0.9048040082522841, 'eval_runtime': 96.8626, 'eval_samples_per_second': 35.029, 'eval_steps_per_second': 4.388, 'epoch': 3.33}
{'loss': 0.9189, 'grad_norm': 3.302363634109497, 'learning_rate': 3.161397670549085e-06, 'epoch': 3.37}
{'loss': 0.905, 'grad_norm': 3.0125815868377686, 'learning_rate': 2.953410981697172e-06, 'epoch': 3.41}
{'loss': 0.9176, 'grad_norm': 4.144485950469971, 'learning_rate': 2.745424292845258e-06, 'epoch': 3.45}
{'loss': 0.9093, 'grad_norm': 2.905592918395996, 'learning_rate': 2.5374376039933445e-06, 'epoch': 3.49}
{'loss': 0.9357, 'grad_norm': 2.771632194519043, 'learning_rate': 2.329450915141431e-06, 'epoch': 3.54}
{'loss': 0.9394, 'grad_norm': 3.7897040843963623, 'learning_rate': 2.1214642262895175e-06, 'epoch': 3.58}
{'loss': 0.9628, 'grad_norm': 3.993272542953491, 'learning_rate': 1.913477537437604e-06, 'epoch': 3.62}
{'loss': 0.8664, 'grad_norm': 1.8706133365631104, 'learning_rate': 1.7054908485856905e-06, 

  0%|          | 0/425 [00:00<?, ?it/s]

{'eval_loss': 0.3991778492927551, 'eval_accuracy': 0.9030356616563513, 'eval_runtime': 94.5027, 'eval_samples_per_second': 35.904, 'eval_steps_per_second': 4.497, 'epoch': 3.74}
{'loss': 0.8987, 'grad_norm': 2.7835891246795654, 'learning_rate': 1.0815307820299502e-06, 'epoch': 3.79}
{'loss': 0.9815, 'grad_norm': 3.472989797592163, 'learning_rate': 8.735440931780367e-07, 'epoch': 3.83}
{'loss': 0.8762, 'grad_norm': 2.797670364379883, 'learning_rate': 6.655574043261231e-07, 'epoch': 3.87}
{'loss': 0.9524, 'grad_norm': 3.567248582839966, 'learning_rate': 4.575707154742097e-07, 'epoch': 3.91}
{'loss': 0.9202, 'grad_norm': 2.5983433723449707, 'learning_rate': 2.495840266222962e-07, 'epoch': 3.95}
{'loss': 0.9054, 'grad_norm': 4.204604148864746, 'learning_rate': 4.1597337770382696e-08, 'epoch': 3.99}
{'train_runtime': 3066.732, 'train_samples_per_second': 25.078, 'train_steps_per_second': 1.568, 'train_loss': 1.441121276424649, 'epoch': 4.0}
***** train metrics *****
  epoch                 

In [19]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

  0%|          | 0/425 [00:00<?, ?it/s]

***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.9048
  eval_loss               =     0.3982
  eval_runtime            = 0:02:02.00
  eval_samples_per_second =     27.811
  eval_steps_per_second   =      3.484


In [34]:
import torch
import torch.nn as nn
num_total_classes = 550

# Assuming you have a pre-existing model with an initial classifier
# Save the original weights and bias
original_weights = model.classifier.weight.data
original_bias = model.classifier.bias.data

# Create a new classifier with the total number of classes
model.classifier = nn.Linear(model.classifier.in_features, num_total_classes)

# Initialize the new classifier
with torch.no_grad():
    # Copy the original weights to the top part of the new weight matrix
    model.classifier.weight.data[:original_weights.shape[0]] = original_weights
    model.classifier.bias.data[:original_bias.shape[0]] = original_bias

    # Optionally, you can initialize the new additional weights 
    # For example, with small random values
    torch.nn.init.xavier_uniform_(model.classifier.weight.data[original_weights.shape[0]:])
    torch.nn.init.zeros_(model.classifier.bias.data[original_bias.shape[0]:])

In [35]:
model.save_pretrained('finetuned-birds-classifier-550-species')


In [42]:
import torch 
import requests 
from torchvision import transforms 
from io import BytesIO 
from PIL import Image 

# Determine the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for the input image 
transform = transforms.Compose([ 
    transforms.Resize((224, 224)),  # Resize to match model's requirement 
    transforms.ToTensor(),  # Convert image to tensor 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize 
]) 

# Example of applying the transformation to a single image 
response = requests.get("https://cdn.britannica.com/03/137403-050-C20EC0B7/Little-owl.jpg") 
image = Image.open(BytesIO(response.content)).convert("RGB")  # Open image and ensure it's in RGB format 
input_tensor = transform(image).unsqueeze(0)  # Add batch dimension 

# Move input tensor to the same device as the model
input_tensor = input_tensor.to(device)

# Move model to the appropriate device
model = model.to(device)

# Ensure correct data type (optional, depends on model)
input_tensor = input_tensor.half()  # or .half() if model supports it

model.eval() 
with torch.no_grad(): 
    outputs = model(input_tensor) 
    logits = outputs.logits 
    predicted_class = torch.argmax(logits, dim=-1).item() 
    predicted_label = model.config.id2label[predicted_class] 
    print(f"Predicted label: {predicted_label}")

Predicted label: Class_157


In [43]:
#Final code 

from datasets import load_dataset
from transformers import (
    AutoImageProcessor, 
    AutoModelForImageClassification, 
    TrainingArguments, 
    Trainer
)
from torchvision.transforms import (
    CenterCrop, Compose, Normalize, 
    RandomHorizontalFlip, RandomResizedCrop, 
    Resize, ToTensor
)
import torch
import numpy as np

# Load the original dataset
ds = load_dataset("imagefolder", data_dir='training_set')
data = ds["train"].train_test_split(test_size=0.15)

# Get original labels
original_labels = data["train"].features["label"].names

# Define the total number of classes (original + new)
num_total_classes = 550

# Create label mappings
label2id = {label: str(i) for i, label in enumerate(original_labels)}
id2label = {str(i): label for i, label in enumerate(original_labels)}

# Add placeholder labels for additional classes
for i in range(len(original_labels), num_total_classes):
    label2id[f"New_Class_{i}"] = str(i)
    id2label[str(i)] = f"New_Class_{i}"

# Load pre-trained model and image processor
model_name_or_path = 'dennisjooo/Birds-Classifier-EfficientNetB2'
feature_extractor = AutoImageProcessor.from_pretrained(model_name_or_path)

# Create transformations
image_size = 224
normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)

train_transforms = Compose([
    RandomResizedCrop(image_size),
    RandomHorizontalFlip(),
    ToTensor(),
    normalize,
])

val_transforms = Compose([
    Resize(image_size),
    CenterCrop(image_size),
    ToTensor(),
    normalize,
])

def preprocess_train(example_batch):
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    example_batch["pixel_values"] = [
        val_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

# Set transformations
train_ds = data["train"]
val_ds = data["test"]
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

# Load model with expanded classifier
model = AutoModelForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=num_total_classes,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

# Initialize additional classifier weights
with torch.no_grad():
    # Preserve original weights
    original_weights = model.classifier.weight.data[:len(original_labels)]
    original_bias = model.classifier.bias.data[:len(original_labels)]
    
    # Reinitialize full classifier
    model.classifier = torch.nn.Linear(
        model.classifier.in_features, 
        num_total_classes
    )
    
    # Copy original weights
    model.classifier.weight.data[:len(original_labels)] = original_weights
    model.classifier.bias.data[:len(original_labels)] = original_bias
    
    # Initialize new weights with Xavier uniform
    torch.nn.init.xavier_uniform_(
        model.classifier.weight.data[len(original_labels):]
    )
    torch.nn.init.zeros_(
        model.classifier.bias.data[len(original_labels):]
    )

# Training arguments
training_args = TrainingArguments(
    'finetuned-birds-classifier-550-species',
    per_device_train_batch_size=16,
    evaluation_strategy="steps",
    num_train_epochs=4,
    fp16=True,
    save_steps=500,
    eval_steps=500,
    logging_steps=50,
    learning_rate=2e-5,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to='tensorboard',
    load_best_model_at_end=True
)

# Metrics computation
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return {'accuracy': (predictions == eval_pred.label_ids).mean()}

# Data collator
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

# Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    data_collator=collate_fn
)

# Train the model
trainer.train()

# Save the model
trainer.save_model('finetuned-birds-classifier-550-species')

Resolving data files:   0%|          | 0/22620 [00:00<?, ?it/s]

Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at dennisjooo/Birds-Classifier-EfficientNetB2 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([525]) in the checkpoint and torch.Size([550]) in the model instantiated
- classifier.weight: found shape torch.Size([525, 1408]) in the checkpoint and torch.Size([550, 1408]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\PF\Projects\env\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/4808 [00:00<?, ?it/s]

{'loss': 6.4797, 'grad_norm': 5.529053688049316, 'learning_rate': 1.979201331114809e-05, 'epoch': 0.04}
{'loss': 6.1483, 'grad_norm': 5.910373210906982, 'learning_rate': 1.9584026622296175e-05, 'epoch': 0.08}
{'loss': 5.837, 'grad_norm': 5.76523494720459, 'learning_rate': 1.9376039933444262e-05, 'epoch': 0.12}
{'loss': 5.5265, 'grad_norm': 5.570156574249268, 'learning_rate': 1.916805324459235e-05, 'epoch': 0.17}
{'loss': 5.1939, 'grad_norm': 5.660665035247803, 'learning_rate': 1.8960066555740436e-05, 'epoch': 0.21}
{'loss': 4.8835, 'grad_norm': 5.580504417419434, 'learning_rate': 1.8752079866888523e-05, 'epoch': 0.25}
{'loss': 4.6051, 'grad_norm': 5.871239185333252, 'learning_rate': 1.8544093178036607e-05, 'epoch': 0.29}
{'loss': 4.3183, 'grad_norm': 5.946713447570801, 'learning_rate': 1.8336106489184694e-05, 'epoch': 0.33}
{'loss': 4.0749, 'grad_norm': 5.71154260635376, 'learning_rate': 1.812811980033278e-05, 'epoch': 0.37}
{'loss': 3.8716, 'grad_norm': 5.569149494171143, 'learning_ra

  0%|          | 0/425 [00:00<?, ?it/s]

...

{'eval_loss': 0.5498448610305786, 'eval_accuracy': 0.8968464485705865, 'eval_runtime': 98.7332, 'eval_samples_per_second': 34.365, 'eval_steps_per_second': 4.305, 'epoch': 2.91}
{'loss': 1.0796, 'grad_norm': 4.114582538604736, 'learning_rate': 5.24126455906822e-06, 'epoch': 2.95}
{'loss': 1.0766, 'grad_norm': 3.5651228427886963, 'learning_rate': 5.033277870216307e-06, 'epoch': 3.0}
{'loss': 1.1, 'grad_norm': 3.208573818206787, 'learning_rate': 4.825291181364393e-06, 'epoch': 3.04}
{'loss': 1.0945, 'grad_norm': 3.526221990585327, 'learning_rate': 4.617304492512479e-06, 'epoch': 3.08}
{'loss': 1.0493, 'grad_norm': 3.9107208251953125, 'learning_rate': 4.409317803660566e-06, 'epoch': 3.12}
{'loss': 1.1119, 'grad_norm': 4.45220422744751, 'learning_rate': 4.201331114808653e-06, 'epoch': 3.16}
{'loss': 1.1393, 'grad_norm': 4.584967136383057, 'learning_rate': 3.993344425956739e-06, 'epoch': 3.2}
{'loss': 1.0492, 'grad_norm': 3.571671962738037, 'learning_rate': 3.7853577371048255e-06, 'epoch': 

  0%|          | 0/425 [00:00<?, ?it/s]

{'eval_loss': 0.5120459794998169, 'eval_accuracy': 0.8921308576480991, 'eval_runtime': 97.4375, 'eval_samples_per_second': 34.822, 'eval_steps_per_second': 4.362, 'epoch': 3.33}
{'loss': 1.0491, 'grad_norm': 3.22322678565979, 'learning_rate': 3.161397670549085e-06, 'epoch': 3.37}
{'loss': 1.0353, 'grad_norm': 4.750710964202881, 'learning_rate': 2.953410981697172e-06, 'epoch': 3.41}
{'loss': 1.0419, 'grad_norm': 3.4684438705444336, 'learning_rate': 2.745424292845258e-06, 'epoch': 3.45}
{'loss': 1.0257, 'grad_norm': 4.880771636962891, 'learning_rate': 2.5374376039933445e-06, 'epoch': 3.49}
{'loss': 1.0237, 'grad_norm': 4.533999443054199, 'learning_rate': 2.329450915141431e-06, 'epoch': 3.54}
{'loss': 1.0751, 'grad_norm': 3.783235549926758, 'learning_rate': 2.1214642262895175e-06, 'epoch': 3.58}
{'loss': 1.0989, 'grad_norm': 3.2126340866088867, 'learning_rate': 1.913477537437604e-06, 'epoch': 3.62}
{'loss': 1.0961, 'grad_norm': 4.03369140625, 'learning_rate': 1.7054908485856905e-06, 'epoc

  0%|          | 0/425 [00:00<?, ?it/s]

{'eval_loss': 0.5090899467468262, 'eval_accuracy': 0.8997936928971412, 'eval_runtime': 97.018, 'eval_samples_per_second': 34.973, 'eval_steps_per_second': 4.381, 'epoch': 3.74}
{'loss': 1.0635, 'grad_norm': 4.326874732971191, 'learning_rate': 1.0815307820299502e-06, 'epoch': 3.79}
{'loss': 1.0724, 'grad_norm': 4.009945869445801, 'learning_rate': 8.735440931780367e-07, 'epoch': 3.83}
{'loss': 1.0586, 'grad_norm': 4.6193952560424805, 'learning_rate': 6.655574043261231e-07, 'epoch': 3.87}
{'loss': 1.1242, 'grad_norm': 3.9450597763061523, 'learning_rate': 4.575707154742097e-07, 'epoch': 3.91}
{'loss': 1.0393, 'grad_norm': 3.596954584121704, 'learning_rate': 2.495840266222962e-07, 'epoch': 3.95}
{'loss': 1.0266, 'grad_norm': 3.776167154312134, 'learning_rate': 4.575707154742097e-08, 'epoch': 3.99}
{'train_runtime': 2941.7217, 'train_samples_per_second': 26.144, 'train_steps_per_second': 1.634, 'train_loss': 1.9344128850295816, 'epoch': 4.0}
